In [18]:
import ccxt
import pandas as pd
import numpy as np

# 创建Binance交易所实例
exchange = ccxt.binance()

# 获取BTC/USDT的历史K线数据，时间间隔为5分钟
symbol = 'BTC/USDT'
timeframe = '5m'
limit = 5000  # 可以根据需要调整数量
ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

# 转换为pandas DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

In [5]:
# 统计5分钟K线的波动幅度分布
df['5m_range'] = df['high'] - df['low']
range_distribution = df['5m_range'].describe()

print("5分钟K线波动幅度分布：")
print(range_distribution)

# 统计某个关键价格（例如移动均线）的突破情况
df['20_SMA'] = df['close'].rolling(window=20).mean()
df['breakout_above_20SMA'] = df['close'] > df['20_SMA']

# 计算突破20SMA后的涨跌幅度
df['returns'] = df['close'].pct_change()
breakout_returns = df.loc[df['breakout_above_20SMA'], 'returns'].mean()

print(f"突破20SMA后平均涨幅为: {breakout_returns:.2%}")

5分钟K线波动幅度分布：
count    500.000000
mean     108.155820
std      101.917989
min       11.910000
25%       50.162500
50%       78.430000
75%      132.990000
max      920.360000
Name: 5m_range, dtype: float64
突破20SMA后平均涨幅为: 0.03%


In [19]:
import ccxt
import pandas as pd
import numpy as np

# 创建Binance交易所实例
exchange = ccxt.binance()

# 获取BTC/USDT的历史K线数据，时间间隔为5分钟
symbol = 'BTC/USDT'
timeframe = '5m'
limit = 1000  # 可以根据需要调整数量
ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

# 转换为pandas DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# 提取日期信息
df['date'] = df['timestamp'].dt.date

# 计算每日的最高价和最低价
daily_highs = df.groupby('date')['high'].max()
daily_lows = df.groupby('date')['low'].min()

# 添加每日最高价和最低价信息
df['daily_high'] = df['date'].map(daily_highs)
df['daily_low'] = df['date'].map(daily_lows)

# 判断每根K线是否为当天最高价或最低价
df['is_daily_high_reached'] = df['high'] >= df['daily_high']
df['is_daily_low_reached'] = df['low'] <= df['daily_low']

# 判断截至当前K线，是否已经达到过当天的最高价或最低价
df['high_or_low_reached'] = df.groupby('date').apply(lambda x: x['is_daily_high_reached'].cummax() | x['is_daily_low_reached'].cummax()).reset_index(level=0, drop=True)

# 计算每根K线后当天最高价或最低价已经出现的概率
probability_by_candle = df.groupby(df.index % (24*12))['high_or_low_reached'].mean() * 100

print("每根5分钟K线后，已出现当天最高价或最低价的概率：")
print(probability_by_candle)

每根5分钟K线后，已出现当天最高价或最低价的概率：
0       75.0
1       75.0
2       75.0
3       75.0
4      100.0
       ...  
283    100.0
284    100.0
285    100.0
286    100.0
287    100.0
Name: high_or_low_reached, Length: 288, dtype: float64


/var/folders/4n/ssdlv9m155j06vdf68lmf_480000gn/T/ipykernel_98661/994396495.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['high_or_low_reached'] = df.groupby('date').apply(lambda x: x['is_daily_high_reached'].cummax() | x['is_daily_low_reached'].cummax()).reset_index(level=0, drop=True)


In [15]:
import ccxt
import pandas as pd

# 创建Binance交易所实例
exchange = ccxt.binance()

# 获取BTC/USDT的历史K线数据，时间间隔为1天
symbol = 'BTC/USDT'
timeframe = '1d'
limit = 1000  # 可以根据需要调整数量
ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

# 转换为pandas DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# 提取周几信息
df['weekday'] = df['timestamp'].dt.day_name()

# 计算每日波动幅度
df['daily_range'] = df['high'] - df['low']

# 按周分组并找出每周波动最大的一天
df['week_start'] = df['timestamp'].dt.to_period('W').apply(lambda r: r.start_time)
weekly_max = df.loc[df.groupby('week_start')['daily_range'].idxmax()]

# 统计每个周几成为波动最大一天的次数
weekday_counts = weekly_max['weekday'].value_counts(normalize=True) * 100

print("每周波动最大的一天是周几的概率：")
print(weekday_counts)

每周波动最大的一天是周几的概率：
weekday
Monday       23.076923
Wednesday    20.279720
Friday       18.881119
Tuesday      18.181818
Thursday     13.286713
Sunday        3.496503
Saturday      2.797203
Name: proportion, dtype: float64
